In [128]:
import glob
import json
import re
from functools import lru_cache
from tabulate import tabulate
from ipdb import set_trace
import pandas as pd

In [133]:
pd.set_option('display.max_colwidth', -1)

In [126]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
regex = re.compile("([0-9]+)(.*)")

In [127]:
EXP=1
SAMPLE="bert-100"

In [83]:
with open(f"../results/{EXP}/predictions_.json") as f:
     preds = json.load(f)

In [84]:
with open(f"../data/{SAMPLE}/dev.json") as f:
    dev_ds = json.load(f)

In [85]:
class Schemas(object):

    def __init__(self, filepath):
        with open(filepath) as f:
            self.index = {}
            for schema in json.load(f):
                service_name = schema["service_name"]
                self.index[service_name] = schema

    @lru_cache(maxsize=None)
    def get(self, service):
        result = dict(
            # service
            name=service,
            desc=self.index[service]["description"],
            
            # slots
            slot_name=[],
            slot_desc=[],
            slot_iscat=[], 
            slot_vals=[], # collected only for cat slots.. not sure if that makes sense

            # intents
            intent_name=[],
            intent_desc=[],
            intent_istrans=[],
            intent_reqslots=[],
            intent_optslots=[],
            intent_optvals=[],
        )

        for slot in self.index[service]["slots"]:
            result["slot_name"].append(slot["name"])
            result["slot_desc"].append(slot["description"])
            result["slot_iscat"].append(slot["is_categorical"])
            result["slot_vals"].append(slot["possible_values"])
        
        for intent in self.index[service]["intents"]:
            result["intent_name"].append(intent["name"])
            result["intent_desc"].append(intent["description"])
            result["intent_istrans"].append(intent["is_transactional"])
            result["intent_reqslots"].append(intent["required_slots"])
            result["intent_optslots"].append(list(intent["optional_slots"].keys()))
            result["intent_optvals"].append(list(intent["optional_slots"].values()))

        return result    
    
    
dev_schema = Schemas("../data/dev/schema.json")

In [142]:
tot_goal_acc = 0
tot_joint_acc = 0
num_dials = 0

for dial in dev_ds["data"]:
    num_dials += 1
    dial_goal_acc = 0
    dial_joint_acc = 0
    num_turns = 1
    
    results = []
    
    for turn in dial["paragraphs"]:
        goal_acc = 1
        count = 1
        
        memory = turn["context"].replace(" . ", "\n")
        
        for qas in turn["qas"]:
            query = qas["question"].replace(" . ", "\n")
            target = qas["answers"][0]["text"]
            pred = preds[qas["id"]].split(" . ")[0]
            
            dialid, turnid, serv, slot = qas["id"].split("/")
            slotid = dev_schema.get(serv)["slot_name"].index(slot)
            slot_iscat = dev_schema.get(serv)["slot_iscat"][slotid]
            
            correct = pred.lower().startswith(target.lower())

            if target != "NONE":
                goal_acc += int(correct)
                count += 1

            if target != "NONE":
                if not correct:
                    dialid = "==> " + dialid
                results.append((dialid, turnid, slot, slot_iscat, memory, query, pred, target, correct))
        
        dial_joint_acc += int((goal_acc / count) == 1)
        dial_goal_acc += goal_acc / count 
        num_turns += 1
    
    tot_goal_acc += dial_goal_acc / num_turns
    tot_joint_acc += dial_joint_acc / num_turns
    num_dials += 1
    
    
    df = pd.DataFrame(results, columns=["dial", "turn", "slot", "iscat", "memory", "query", "pred", "target", "correct"])
    display(HTML(df.to_html().replace("\\n", "<br>")))
    
print(tot_goal_acc/num_dials)
print(tot_joint_acc/num_dials)

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00000,0,time,False,NONEdontcare2019-03-01half past 11 in the morning,this is turn number 0dialogue startedI want to make a restaurant reservation for 2 people at half past 11 in the morning.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,half past 11 in the morning,False
1,1_00000,0,number_of_seats,True,NONEdontcare123456,this is turn number 0dialogue startedI want to make a restaurant reservation for 2 people at half past 11 in the morning.A popular restaurant search and reservation serviceNumber of seats to reserve at the restaurantMake a table reservation at a restaurantintent is transactionalslot is categorical,2,2,True
2,1_00000,2,restaurant_name,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino,this is turn number 2What city do you want to dine in? Do you have a preferred restaurant?Please find restaurants in San Jose. Can you try Sino?A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,Sino,Sino,True
3,==> 1_00000,2,time,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino,this is turn number 2What city do you want to dine in? Do you have a preferred restaurant?Please find restaurants in San Jose. Can you try Sino?A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,half past 11 in the morning,False
4,1_00000,2,location,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino,this is turn number 2What city do you want to dine in? Do you have a preferred restaurant?Please find restaurants in San Jose. Can you try Sino?A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Jose,San Jose,True
5,1_00000,2,number_of_seats,True,NONEdontcare123456,this is turn number 2What city do you want to dine in? Do you have a preferred restaurant?Please find restaurants in San Jose. Can you try Sino?A popular restaurant search and reservation serviceNumber of seats to reserve at the restaurantMake a table reservation at a restaurantintent is transactionalslot is categorical,2,2,True
6,==> 1_00000,4,date,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino11:30 amtoday,"this is turn number 4Confirming: I will reserve a table for 2 people at Sino in San Jose. The reservation time is 11:30 am today.Yes, thanks. What's their phone number?A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",Sino,today,False
7,1_00000,4,restaurant_name,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino11:30 amtoday,"this is turn number 4Confirming: I will reserve a table for 2 people at Sino in San Jose. The reservation time is 11:30 am today.Yes, thanks. What's their phone number?A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical",Sino,Sino,True
8,==> 1_00000,4,time,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino11:30 amtoday,"this is turn number 4Confirming: I will reserve a table for 2 people at Sino in San Jose. The reservation time is 11:30 am today.Yes, thanks. What's their phone number?A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",Sino,11:30 am,False
9,1_00000,4,location,False,NONEdontcare2019-03-01half past 11 in the morningSan JoseSino11:30 amtoday

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,1_00001,2,restaurant_name,False,NONEdontcare2019-03-0111:30SaratogaSipan,this is turn number 2Which area would you like me to look in? Which restaurant would you like to eat in and at what time?Look for a restaurant in Saratoga. Check to see if I can have a table for 1 at Sipan for 11:30.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,Sipan,Sipan,True
1,1_00001,2,time,False,NONEdontcare2019-03-0111:30SaratogaSipan,this is turn number 2Which area would you like me to look in? Which restaurant would you like to eat in and at what time?Look for a restaurant in Saratoga. Check to see if I can have a table for 1 at Sipan for 11:30.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,11:30,11:30,True
2,==> 1_00001,2,location,False,NONEdontcare2019-03-0111:30SaratogaSipan,this is turn number 2Which area would you like me to look in? Which restaurant would you like to eat in and at what time?Look for a restaurant in Saratoga. Check to see if I can have a table for 1 at Sipan for 11:30.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,11:30,Saratoga,False
3,==> 1_00001,2,number_of_seats,True,NONEdontcare123456,this is turn number 2Which area would you like me to look in? Which restaurant would you like to eat in and at what time?Look for a restaurant in Saratoga. Check to see if I can have a table for 1 at Sipan for 11:30.A popular restaurant search and reservation serviceNumber of seats to reserve at the restaurantMake a table reservation at a restaurantintent is transactionalslot is categorical,NONE,1,False
4,==> 1_00001,4,date,False,NONEdontcare2019-03-0111:30SaratogaSipanSipan Peruvian Restaurant & Bar11:30 amtodayMarch 2ndRosie Mccann's,this is turn number 4You would like a table for 1 person for today at 11:30 am at the Sipan Peruvian Restaurant & Bar in Saratoga?I would like to book at table at Rosie Mccann's for March 2nd.A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,Sipan Peruvian Restaurant & Bar,March 2nd,False
5,1_00001,4,restaurant_name,False,NONEdontcare2019-03-0111:30SaratogaSipanSipan Peruvian Restaurant & Bar11:30 amtodayMarch 2ndRosie Mccann's,this is turn number 4You would like a table for 1 person for today at 11:30 am at the Sipan Peruvian Restaurant & Bar in Saratoga?I would like to book at table at Rosie Mccann's for March 2nd.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,Rosie Mccann's,Rosie Mccann's,True
6,1_00001,4,time,False,NONEdontcare2019-03-0111:30SaratogaSipanSipan Peruvian Restaurant & Bar11:30 amtodayMarch 2ndRosie Mccann's,this is turn number 4You would like a table for 1 person for today at 11:30 am at the Sipan Peruvian Restaurant & Bar in Saratoga?I would like to book at table at Rosie Mccann's for March 2nd.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,11:30,11:30,True
7,1_00001,4,location,False,NONEdontcare2019-03-0111:30SaratogaSipanSipan Peruvian Restaurant & Bar11:30 amtodayMarch 2ndRosie Mccann's,this is turn number 4You would like a table for 1 person for today at 11:30 am at the Sipan Peruvian Restaurant & Bar in Saratoga?I would like to book at table at Rosie Mccann's for March 2nd.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantin

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00002,0,restaurant_name,False,NONEdontcare2019-03-01Bourbon Steak,"this is turn number 0dialogue startedI want to reserve a table at a restaurant, specifically Bourbon Steak.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical",NONE,Bourbon Steak,False
1,==> 1_00002,2,restaurant_name,False,NONEdontcare2019-03-01Bourbon SteakSan Francisco,this is turn number 2Which location of Bourbon Steak do you want to save a table?Find Bourbon Steaks in San Francisco please.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,Bourbon Steak,False
2,1_00002,2,location,False,NONEdontcare2019-03-01Bourbon SteakSan Francisco,this is turn number 2Which location of Bourbon Steak do you want to save a table?Find Bourbon Steaks in San Francisco please.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Francisco,San Francisco,True
3,==> 1_00002,4,restaurant_name,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoon,this is turn number 4What time do you want to book the table for?Please do it for one in the afternoon.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,Bourbon Steak,False
4,1_00002,4,time,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoon,this is turn number 4What time do you want to book the table for?Please do it for one in the afternoon.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,one in the afternoon,one in the afternoon,True
5,1_00002,4,location,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoon,this is turn number 4What time do you want to book the table for?Please do it for one in the afternoon.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Francisco,San Francisco,True
6,1_00002,6,date,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoonBourbon Steak Restaurant1 pmtoday,"this is turn number 6Okay. Just to be clear, you want a table at Bourbon Steak Restaurant in San Francisco for 2 people today at 1 pm.Yes, that is correct.A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",today,today,True
7,1_00002,6,restaurant_name,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoonBourbon Steak Restaurant1 pmtoday,"this is turn number 6Okay. Just to be clear, you want a table at Bourbon Steak Restaurant in San Francisco for 2 people today at 1 pm.Yes, that is correct.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical",Bourbon Steak,Bourbon Steak,True
8,1_00002,6,time,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoonBourbon Steak Restaurant1 pmtoday,"this is turn number 6Okay. Just to be clear, you want a table at Bourbon Steak Restaurant in San Francisco for 2 people today at 1 pm.Yes, that is correct.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",1 pm,1 pm,True
9,1_00002,6,location,False,NONEdontcare2019-03-01Bourbon SteakSan Franciscoone in the afternoonBourbon Steak Rest

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00003,0,time,False,NONEdontcare2019-03-01morning 11:15,this is turn number 0dialogue startedI would like to make a restaurant reservation at morning 11:15.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,morning 11:15,False
1,==> 1_00003,2,restaurant_name,False,NONEdontcare2019-03-01morning 11:15Andes Cafe,this is turn number 2Which restaurant do you want to go to?Can I get a reservation at Andes Cafe?A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,Andes Cafe,False
2,==> 1_00003,2,time,False,NONEdontcare2019-03-01morning 11:15Andes Cafe,this is turn number 2Which restaurant do you want to go to?Can I get a reservation at Andes Cafe?A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,morning 11:15,False
3,==> 1_00003,4,restaurant_name,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan Mateo,this is turn number 4Which location do you want to get the reservation at?Find a restaurant in San Mateo.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,Andes Cafe,False
4,==> 1_00003,4,time,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan Mateo,this is turn number 4Which location do you want to get the reservation at?Find a restaurant in San Mateo.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,morning 11:15,False
5,1_00003,4,location,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan Mateo,this is turn number 4Which location do you want to get the reservation at?Find a restaurant in San Mateo.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Mateo,San Mateo,True
6,1_00003,6,restaurant_name,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan MateoAndes Cafe San Mateo11:15 amtoday,this is turn number 6Please confirm you want to make a reservation for 2 people today at 11:15 am at Andes Cafe San Mateo in San Mateo.The reservation is just for 1 person.A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,Andes Cafe,Andes Cafe,True
7,==> 1_00003,6,time,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan MateoAndes Cafe San Mateo11:15 amtoday,this is turn number 6Please confirm you want to make a reservation for 2 people today at 11:15 am at Andes Cafe San Mateo in San Mateo.The reservation is just for 1 person.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,Andes Cafe San Mateo,11:15 am,False
8,1_00003,6,location,False,NONEdontcare2019-03-01morning 11:15Andes CafeSan MateoAndes Cafe San Mateo11:15 amtoday,this is turn number 6Please confirm you want to make a reservation for 2 people today at 11:15 am at Andes Cafe San Mateo in San Mateo.The reservation is just for 1 person.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Mateo,San Mateo,True
9,==> 1_00003,6,number_of_seats,True,NONEdontcare123456,this is turn number 6Please confirm you want to make a reservation for 2 people today at 11:15 am at Andes Cafe San Mateo in San Mateo.The reservation is just for 

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00004,0,date,False,NONEdontcare2019-03-01March 5th,this is turn number 0dialogue startedI want to make a dinner reservation on March 5th.A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,NONE,March 5th,False
1,1_00004,2,date,False,NONEdontcare2019-03-01March 5th18:30San Jose,"this is turn number 2What area are you looking in? What time do you want the reservation for?I want a restaurant in San Jose, for 18:30, please.A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",March 5th,March 5th,True
2,1_00004,2,time,False,NONEdontcare2019-03-01March 5th18:30San Jose,"this is turn number 2What area are you looking in? What time do you want the reservation for?I want a restaurant in San Jose, for 18:30, please.A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",18:30,18:30,True
3,1_00004,2,location,False,NONEdontcare2019-03-01March 5th18:30San Jose,"this is turn number 2What area are you looking in? What time do you want the reservation for?I want a restaurant in San Jose, for 18:30, please.A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical",San Jose,San Jose,True
4,1_00004,4,date,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table,this is turn number 4Which restaurant do you want to book a table at?Can you get me a table at Ludwig's German Table?A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,March 5th,March 5th,True
5,1_00004,4,restaurant_name,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table,this is turn number 4Which restaurant do you want to book a table at?Can you get me a table at Ludwig's German Table?A popular restaurant search and reservation serviceName of the restaurantMake a table reservation at a restaurantintent is transactionalslot is not categorical,Ludwig's German Table,Ludwig's German Table,True
6,1_00004,4,time,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table,this is turn number 4Which restaurant do you want to book a table at?Can you get me a table at Ludwig's German Table?A popular restaurant search and reservation serviceTentative time of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical,18:30,18:30,True
7,1_00004,4,location,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table,this is turn number 4Which restaurant do you want to book a table at?Can you get me a table at Ludwig's German Table?A popular restaurant search and reservation serviceCity where the restaurant is locatedMake a table reservation at a restaurantintent is transactionalslot is not categorical,San Jose,San Jose,True
8,1_00004,6,date,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table6:30 pmnext Tuesday,"this is turn number 6So you want a table for 2 people next Tuesday at 6:30 pm at Ludwig's German Table in San Jose?Yes, that's right.A popular restaurant search and reservation serviceTentative date of restaurant reservationMake a table reservation at a restaurantintent is transactionalslot is not categorical",March 5th,March 5th,True
9,1_00004,6,restaurant_name,False,NONEdontcare2019-03-01March 5th18:30San JoseLudwig's German Table6:30 pmnext Tuesday,"this is turn number 6So you want a table for 2 people next Tuesday at 6:30 pm at Ludwig's German Table in San Jose?Yes, that's right.A popular restaura

...

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,1_00096,2,destination_city,False,NONEdontcareChicago,this is turn number 2Where will you be travelling to?My destination is Chicago.Find one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Chicago,Chicago,True
1,1_00096,4,origin_city,False,NONEdontcareChicagoSydney12th of this month,this is turn number 4Please tell me your departure city and your preferred date of travel.Please find flights departing from Sydney. I'd like Economy class tickets on a flight leaving on 12th of this month.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Sydney,Sydney,True
2,==> 1_00096,4,departure_date,False,NONEdontcareChicagoSydney12th of this month,this is turn number 4Please tell me your departure city and your preferred date of travel.Please find flights departing from Sydney. I'd like Economy class tickets on a flight leaving on 12th of this month.Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,12th of this month,False
3,1_00096,4,destination_city,False,NONEdontcareChicagoSydney12th of this month,this is turn number 4Please tell me your departure city and your preferred date of travel.Please find flights departing from Sydney. I'd like Economy class tickets on a flight leaving on 12th of this month.Find one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Chicago,Chicago,True
4,==> 1_00096,4,flight_class,True,NONEdontcareEconomyPremium EconomyBusinessFirst Class,this is turn number 4Please tell me your departure city and your preferred date of travel.Please find flights departing from Sydney. I'd like Economy class tickets on a flight leaving on 12th of this month.Find one way and round trip flights to your favorite cityFare class of flight bookingSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,Economy,False
5,1_00096,6,origin_city,False,NONEdontcareChicagoSydney12th of this month3:55 pm$405,"this is turn number 6I've found one 1 flight you may like. Would you consider taking a Delta Airlines flight with 1 layover, taking off at 3:55 pm local time, at a cost of $405 per passenger?That sounds good, but I've just been informed that my team's expertise will be required on an extension of the just concluded project, and therefore I need to accompany 2 other team members to and from Sydney. Please find me three seats on a round trip flight.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for round-trip flights to the destination of choiceintent is not transactionalslot is not categorical",Sydney,Sydney,True
6,==> 1_00096,6,departure_date,False,NONEdontcareChicagoSydney12th of this month3:55 pm$405,"this is turn number 6I've found one 1 flight you may like. Would you consider taking a Delta Airlines flight with 1 layover, taking off at 3:55 pm local time, at a cost of $405 per passenger?That sounds good, but I've just been informed that my team's expertise will be required on an extension of the just concluded project, and therefore I need to accompany 2 other team members to and from Sydney. Please find me three seats on a round trip flight.Find one way and round trip flights to your favorite cityDate of departure flightSearch for round-trip flights to the destination of choiceintent is not transactionalslot is not categorical",Sydney,12th of this month,False
7,1_00096,6,destination_city,False,NONEdontcareChicagoSydney12th of th

,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00097,0,departure_date,False,NONEdontcare2nd of March,this is turn number 0dialogue startedCan you help me find 3 tickets for a one way flight for the 2nd of March?Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,2nd of March,False
1,==> 1_00097,0,passengers,True,NONEdontcare1234,this is turn number 0dialogue startedCan you help me find 3 tickets for a one way flight for the 2nd of March?Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,3,False
2,==> 1_00097,2,destination_city,False,NONEdontcare2nd of MarchWashington,this is turn number 2What is the destination?Washington and I want to fly with Delta Airlines.Find one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,2nd of March,Washington,False
3,==> 1_00097,2,departure_date,False,NONEdontcare2nd of MarchWashington,this is turn number 2What is the destination?Washington and I want to fly with Delta Airlines.Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,2nd of March,False
4,==> 1_00097,2,passengers,True,NONEdontcare1234,this is turn number 2What is the destination?Washington and I want to fly with Delta Airlines.Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,2,3,False
5,1_00097,2,airlines,True,NONEdontcareUnited AirlinesAmerican AirlinesDelta AirlinesSouthwest AirlinesAlaska AirlinesBritish AirwaysAir CanadaAir France,this is turn number 2What is the destination?Washington and I want to fly with Delta Airlines.Find one way and round trip flights to your favorite cityName of airline operating the flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,Delta Airlines,Delta Airlines,True
6,==> 1_00097,4,origin_city,False,NONEdontcare2nd of MarchWashingtonAtlanta,this is turn number 4What is departure city?Departure city is Atlanta.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Washington,Atlanta,False
7,==> 1_00097,4,departure_date,False,NONEdontcare2nd of MarchWashingtonAtlanta,this is turn number 4What is departure city?Departure city is Atlanta.Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,2nd of March,False
8,1_00097,4,destination_city,False,NONEdontcare2nd of MarchWashingtonAtlanta,this is turn number 4What is departure city?Departure city is Atlanta.Find one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Washington,Washington,True
9,==> 1_00097,4,passengers,True,NONEdontcare1234,this is turn number 4What is departure city?Departure city is Atlanta.Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,3,False


,dial,turn,slot,iscat,memory,query,pred,target,correct
0,1_00098,2,origin_city,False,NONEdontcarePhoenix,"this is turn number 2Which city do you want to depart from?I would like 4 seats, departing from Phoenix.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical",Phoenix,Phoenix,True
1,==> 1_00098,2,passengers,True,NONEdontcare1234,"this is turn number 2Which city do you want to depart from?I would like 4 seats, departing from Phoenix.Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical",2,4,False
2,1_00098,4,origin_city,False,NONEdontcarePhoenixthe 2nd,this is turn number 4What date would you like to travel?I plan to fly out on the 2nd.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Phoenix,Phoenix,True
3,==> 1_00098,4,departure_date,False,NONEdontcarePhoenixthe 2nd,this is turn number 4What date would you like to travel?I plan to fly out on the 2nd.Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,the 2nd,False
4,==> 1_00098,4,passengers,True,NONEdontcare1234,this is turn number 4What date would you like to travel?I plan to fly out on the 2nd.Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,4,False
5,1_00098,6,origin_city,False,NONEdontcarePhoenixthe 2ndSeattle,this is turn number 6What is the planned destination?I am flying to Seattle.Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Phoenix,Phoenix,True
6,==> 1_00098,6,departure_date,False,NONEdontcarePhoenixthe 2ndSeattle,this is turn number 6What is the planned destination?I am flying to Seattle.Find one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,the 2nd,False
7,1_00098,6,destination_city,False,NONEdontcarePhoenixthe 2ndSeattle,this is turn number 6What is the planned destination?I am flying to Seattle.Find one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Seattle,Seattle,True
8,==> 1_00098,6,passengers,True,NONEdontcare1234,this is turn number 6What is the planned destination?I am flying to Seattle.Find one way and round trip flights to your favorite cityNumber of passengers to find flight seats forSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,2,4,False
9,1_00098,8,origin_city,False,NONEdontcarePhoenixthe 2ndSeattle10:45 am$224,this is turn number 8I found a flight on American Airlines with 1 layover that departs at 10:45 am and costs $224 per ticket.Can you please find me another flight?Find one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Phoenix,Phoenix,True


,dial,turn,slot,iscat,memory,query,pred,target,correct
0,==> 1_00099,0,departure_date,False,NONEdontcare6th of March,this is turn number 0dialogue startedHelp me find a one way economy flight on the 6th of MarchFind one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,6th of March,False
1,==> 1_00099,0,flight_class,True,NONEdontcareEconomyPremium EconomyBusinessFirst Class,this is turn number 0dialogue startedHelp me find a one way economy flight on the 6th of MarchFind one way and round trip flights to your favorite cityFare class of flight bookingSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,Economy,False
2,==> 1_00099,2,departure_date,False,NONEdontcare6th of MarchPhoenixNew York,this is turn number 2Where are you headed? And where are you leaving from?I'm leaving from Phoenix and going to New YorkFind one way and round trip flights to your favorite cityDate of departure flightSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,NONE,6th of March,False
3,==> 1_00099,2,destination_city,False,NONEdontcare6th of MarchPhoenixNew York,this is turn number 2Where are you headed? And where are you leaving from?I'm leaving from Phoenix and going to New YorkFind one way and round trip flights to your favorite cityCity in which the journey endsSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Phoenix,New York,False
4,1_00099,2,origin_city,False,NONEdontcare6th of MarchPhoenixNew York,this is turn number 2Where are you headed? And where are you leaving from?I'm leaving from Phoenix and going to New YorkFind one way and round trip flights to your favorite cityCity in which the journey originatesSearch for one-way flights to the destination of choiceintent is not transactionalslot is not categorical,Phoenix,Phoenix,True
5,==> 1_00099,2,flight_class,True,NONEdontcareEconomyPremium EconomyBusinessFirst Class,this is turn number 2Where are you headed? And where are you leaving from?I'm leaving from Phoenix and going to New YorkFind one way and round trip flights to your favorite cityFare class of flight bookingSearch for one-way flights to the destination of choiceintent is not transactionalslot is categorical,NONE,Economy,False
6,1_00099,4,origin_city,False,NONEdontcare6th of MarchPhoenixNew York4:10 am$3118th of March,"this is turn number 4How about flying on a Southwest Airlines flight at 4:10 am with 1 layover and costing $311I want to find a round trip instead, make my return the 8th of MarchFind one way and round trip flights to your favorite cityCity in which the journey originatesSearch for round-trip flights to the destination of choiceintent is not transactionalslot is not categorical",Phoenix,Phoenix,True
7,1_00099,4,return_date,False,NONEdontcare6th of MarchPhoenixNew York4:10 am$3118th of March,"this is turn number 4How about flying on a Southwest Airlines flight at 4:10 am with 1 layover and costing $311I want to find a round trip instead, make my return the 8th of MarchFind one way and round trip flights to your favorite cityDate of return flightSearch for round-trip flights to the destination of choiceintent is not transactionalslot is not categorical",8th of March,8th of March,True
8,1_00099,4,departure_date,False,NONEdontcare6th of MarchPhoenixNew York4:10 am$3118th of March,"this is turn number 4How about flying on a Southwest Airlines flight at 4:10 am with 1 layover and costing $311I want to find a round trip instead, make my return the 8th of MarchFind one way and round trip flights to your favorite cityDate of departure flightSearch for round-trip flights to the destination of choiceintent is not transactionalslot is not categorical",6th of March,6th of March,True
9,==> 1_00099,4,destination_city,False,NONEdontcare6th o

0.38483516483792485
0.26847778830413394
